In [1]:
import mlflow.keras
import mlflow
import mlflow.tensorflow
from sklearn.model_selection import train_test_split, cross_val_score

from table_functions import *
from analyze_visualisation import *
from ml_models.decision_tree import *
from ml_models.random_forest import *
from ml_models.xg_boost import *
from ml_models.neural_network_train import *
from ml_models.extra_random_forest import *

In [ ]:
# TO MUSI BYC USTAWIONE ZAWSZE TAK SAMO, a potem jak sie chce umieszczac dane modele w innym folderze to zmienia sie nazwe  artifact_directory = "other_classification"
# jak chce sie wlaczyc potem mlflow w przegladarce, trzeba w terminalu przejsc do lokalizacji 'src' i wpisac komende mlflow ui
ml_runs_directory = r"C:\Users\dlxpmx8\Desktop\Projekt_AI\meb_process_data_analysis\src\mlruns"
mlflow.set_tracking_uri(f"file://{ml_runs_directory}")

In [ ]:
data_keys = ['x_train', 'x_valid', 'x_test', 'y_train', 'y_valid', 'y_test']
ml_data = {key: None for key in data_keys}
for key in ml_data:
    ml_data[key] = read_csv(f'{key}.csv')

In [ ]:
ml_data_no_cat = ml_data.copy()
categorical_columns_ = list(ml_data_no_cat['x_train'].iloc[:, 130:].columns)
for cold in ['x_train', 'x_valid', 'x_test']:
    ml_data_no_cat[cold].drop(columns=categorical_columns_, inplace=True)

<h4>Drzewo decyzyjne</h4>

In [ ]:
clf = decision_tree_model(*ml_data.values(), run_name_="normal_run")
#print_stats(clf, ml_data['x_test'], ml_data['y_test'])

<h4>Las losowy<h4>

In [ ]:
clf = random_forest_model(*ml_data.values(), run_name_="normal_run")

<h4>XGBoost</h4>

In [ ]:
clf = xgb_model(*ml_data.values(), run_name_="normal_run")

<h4>Sieci neuronowe<h4>

In [ ]:
compile_fit_evaluate_model(*ml_data.values(), epochs_=10, batch_size_=64, comment="standarized data", run_name_=f"after_error")

<h4>Super drzewa losowe</h4>

In [ ]:
clf = extra_random_trees(*ml_data.values(), run_name_="first_run")

<h4>Cross Validacja</h4>

Według mnie na razie zostawmy cross validacje w spokoju, na normalnym zbiorze znajdzmy najlepsze parametry dla modeli a potem dopiero zrobmy na takich parametrach cross validacje by zobaczyc 'prawdziwa' jakosc modelu
PS. ponizszy kod wykonuje sie jakas 1h
PS2. dane sie zapisaly ale jakis syf

In [ ]:
artifact_directory="cv_random_forest"
mlflow.set_experiment(artifact_directory)
mlflow.sklearn.autolog()

with start_run(run_name="testowy run"):
    clf = RandomForestClassifier(random_state=0)
    scores = cross_val_score(estimator=clf, X=ml_data['x_train'], y=ml_data['y_train'], cv=5)
print(scores)
# [0.99786842 0.99779825 0.99742105 0.99770175 0.99797368] - wynik